# Definition of a KB system for the Attolab facility

## Usual imports

In [1]:
import ctypes
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
from pyoptix.classes import (Beamline, GaussianSource, RecordingMode, 
                             ConicCylindricalMirror, PlaneFilm, PlaneMirror)


intialzing SR library
OptiX library initialized


In [2]:
hc = h*c/eV
pyoptix.output_notebook()

Loading BokehJS ...

## Definition of the optical elements

Note: the name of the object and its attribute "name" do not need to match, but this can lead to confusion in loading of the beamline later.
See example in [Atto3b](KB_HELIMAG_tolerancing.ipynb)

In [3]:
Bl = Beamline(name="KB")
source = GaussianSource(name="gaussSource")
plan = PlaneMirror(name="M1")
VFM = ConicCylindricalMirror(name="VFM")
HFM = ConicCylindricalMirror(name="HFM")
plan_foc = PlaneFilm(name="foc")

## Setting the elements parameters
Parameters can be set with their value or a dictionnary mapping some or all of theirs underlying field. See documentation for details.

In [4]:
source.nrays = 10000
inverse_e_square = 100e-6 #m
source_fwhm = inverse_e_square/2/0.8493218
source_rms = source_fwhm/2.35
source.sigma_x = source_rms #m
source.sigma_y = source_rms #m
source.sigma_x_div = 1e-3 # rad
source.sigma_y_div = 1e-3 # rad
print(source_fwhm,source_rms)

5.88705011457377e-05 2.5051277083292638e-05


In [5]:
p = 1.8 #m
q = 0.06 #m
theta = 20*degree
dist = 0.05 #m

In [6]:
plan.distance_from_previous = p - dist
plan.theta = theta # rad
plan.phi = 180*degree # rad
plan.next = VFM
plan.recording_mode = RecordingMode.recording_input

In [7]:
VFM.distance_from_previous = dist
VFM.phi = -180*degree # rad
VFM.theta = theta # rad
VFM.inverse_p = -1/(p) # m-1
VFM.inverse_q = 1/(q + dist) # m-1
VFM.theta0 = theta #rad
VFM.next = HFM
VFM.recording_mode = RecordingMode.recording_input

In [8]:
HFM.distance_from_previous = dist
HFM.phi = 90*degree # rad
HFM.theta = theta # rad
HFM.inverse_p = -1/(p + dist) # m-1
HFM.inverse_q = 1/(q) # m-1
HFM.theta0 = theta #rad
HFM.next = plan_foc
HFM.recording_mode = RecordingMode.recording_input

In [9]:
plan_foc.recording_mode = RecordingMode.recording_input
plan_foc.phi = -90*degree # rad
plan_foc.theta= 0*degree
plan_foc.distance_from_previous = q

In [10]:
Bl.chains["whole KB"] = [source, plan, VFM, HFM, plan_foc]
Bl.active_chain = "whole KB"

Chaîne whole KB:
	gaussSource -> M1 -> VFM -> HFM -> foc 


## Verification of the parameters values and saving this configuration in file for later reset

In [11]:
VFM.dump_properties()
HFM.dump_properties()

_= plan_foc.dump_properties()

Dump of all optix parameter for VFM
	 DX: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 DY: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 DZ: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 Dphi: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 Dpsi: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 Dtheta: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 distance: 0.05 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 invp: -0.5555555555555556 [0.0, 0.0],x1.0, type -1, groupe 1, flags 0
	 invq: 9.090909090909092 [0.0, 0.0],x1.0, type -1, groupe 1, flags 0
	 phi: -3.141592653589793 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 psi: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 theta: 0.3490658503988659 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 theta0: 0.3490658503988659 [0.0, 0.0],x1.0, type 1, groupe 1, flags 0
Dump of all optix parameter for HFM
	 DX: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 DY: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 DZ: 0.0 [0.0, 0.0],x1.0, t

In [12]:
initial_configuration = Bl.save_configuration(filename="KB_HELIMAG_initial_config")

## Running the simulation and displaying the result

In [13]:
Bl.clear_impacts(clear_source=True)
Bl.align(20e-9)

Bl.generate(20e-9)

Bl.radiate()

1

Bl.draw_active_chain()

In [14]:
plan_foc.show_diagram(distance_from_oe=0, orthonorm=True)

KB whole KB


({'xy': ColumnDataSource(id='1003', ...),
  'xxp': ColumnDataSource(id='1004', ...),
  'yyp': ColumnDataSource(id='1005', ...)},
 [Column(id='1163', ...), Column(id='1321', ...), Column(id='1479', ...)])

In [15]:
diag = plan_foc.get_diagram()
print(diag["X"].std())
print(diag["Y"].std())
print(diag["dX"].std())
print(diag["dY"].std())

8.149897498925234e-07
1.5418607630884858e-06
0.03093396763056067
0.016292620632775638


### Footprint calculation

The spot diagram on a surface is actually measured in a plane normal to the beam and centered at the center of the optical element.

Therefore an estimation of the footprint (which neglects surface curvature) can be done by manually distorting the spot diagram

In [16]:
HFM.show_diagram(distance_from_oe=0, display="xy", orthonorm=True)
footprint_HFM = HFM.get_diagram()
p = pyoptix.ui_objects.figure()
p.scatter(footprint_HFM["X"], footprint_HFM["Y"]/np.sin(theta))
pyoptix.ui_objects.show(p)

KB whole KB


In [17]:
VFM.show_diagram(distance_from_oe=0, display="xy", orthonorm=True)
footprint_VFM = VFM.get_diagram()
p = pyoptix.ui_objects.figure()
p.scatter(footprint_VFM["X"], footprint_VFM["Y"]/np.sin(theta))
pyoptix.ui_objects.show(p)

KB whole KB


## Reloading the initial configuration and trying optimizing screen distance

In [18]:
Bl.load_configuration(filename="KB_HELIMAG_initial_config")

In [19]:
from pyoptix.optimize import find_focus
opt_dist = find_focus(Bl, plan_foc, 20e-9)
print(opt_dist)

Minimization success: True, converged to a distance of [0.]
0.0


## Saving the beamline for using in another notebook

In [20]:
pyoptix.save_beamline(Bl, "whole KB", "KB_HELIMAG_definition.pickle")